<a href="https://colab.research.google.com/github/Digital-AI-Finance/Digital-Finance-Introduction/blob/main/day_04/notebooks/NB10_Stablecoin_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NB10: Stablecoin Stability Analysis

**Topic:** 4.3 - Stablecoins and Programmable Finance

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand Stablecoin Mechanisms**: Compare fiat-backed, crypto-backed, and algorithmic stablecoin designs
2. **Analyze Peg Stability**: Measure how often and how much stablecoins deviate from their $1 target
3. **Simulate Collateralized Stablecoins**: Build a working model of a DAI-like crypto-backed stablecoin
4. **Understand De-peg Risks**: Explore liquidation cascades and systemic risk scenarios
5. **Study Historical Failures**: Analyze the Terra/UST collapse as a case study in algorithmic stablecoin risk

### Data Analysis Primer: Understanding pandas and NumPy

This notebook uses two key Python libraries for data analysis:

**NumPy** - Numerical computing library
- `np.random.seed(42)` → Makes random numbers reproducible
- `np.linspace(start, end, n)` → Creates n evenly-spaced values
- `np.random.normal(mean, std)` → Generates random numbers from a bell curve
- `np.mean()`, `np.std()` → Calculate average and standard deviation

**pandas** - Data manipulation library
- `pd.DataFrame()` → Creates a table of data (like Excel)
- `df.to_string()` → Converts table to readable text
- `pd.date_range()` → Creates a sequence of dates

**matplotlib** - Plotting library
- `plt.plot()` → Line charts
- `plt.bar()` → Bar charts
- `plt.hist()` → Histograms (distribution)
- `plt.axhline()` → Horizontal reference line

**Don't worry if you're new to these!** The code is commented to explain each step. Focus on understanding the financial concepts - the code is just a tool to visualize them.

## Section 1: Setup

We'll use standard Python libraries for simulation and analysis. No external blockchain connections required.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib for better plots
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## Section 2: Types of Stablecoins

Stablecoins are cryptocurrencies designed to maintain a stable value, typically pegged to $1 USD. There are three main types:

### 1. Fiat-Backed (Centralized)
- **Examples**: USDT (Tether), USDC (Circle)
- **Mechanism**: Each token is backed 1:1 by USD reserves held by a central custodian
- **Pros**: Simple, high stability, transparent audits (USDC)
- **Cons**: Centralized, requires trust, can be frozen/blacklisted

### 2. Crypto-Backed (Decentralized)
- **Examples**: DAI (MakerDAO), LUSD (Liquity)
- **Mechanism**: Over-collateralized by crypto assets (e.g., 150% collateral ratio)
- **Pros**: Decentralized, transparent, censorship-resistant
- **Cons**: Capital inefficient, liquidation risk, complex

### 3. Algorithmic (Experimental)
- **Examples**: UST (Terra - failed), FRAX (hybrid)
- **Mechanism**: Maintain peg through algorithmic supply adjustments
- **Pros**: Capital efficient, no collateral needed
- **Cons**: High risk of death spiral, untested models, many have failed

In [ ]:
# Define stablecoin characteristics
stablecoin_types = {
    'Fiat-Backed': {
        'examples': ['USDT', 'USDC', 'BUSD', 'TUSD'],
        'collateral': 'USD reserves, T-bills, commercial paper',
        'collateral_ratio': 1.0,  # 100%
        'decentralization': 'Centralized',
        'capital_efficiency': 'Low (1:1)',
        'risk_factors': ['Counterparty risk', 'Regulatory risk', 'Censorship'],
        'typical_deviation': 0.001  # 0.1% typical deviation
    },
    'Crypto-Backed': {
        'examples': ['DAI', 'LUSD', 'sUSD', 'MIM'],
        'collateral': 'ETH, WBTC, other crypto assets',
        'collateral_ratio': 1.5,  # 150% minimum
        'decentralization': 'Decentralized',
        'capital_efficiency': 'Medium (requires over-collateralization)',
        'risk_factors': ['Collateral volatility', 'Liquidation cascades', 'Smart contract risk'],
        'typical_deviation': 0.005  # 0.5% typical deviation
    },
    'Algorithmic': {
        'examples': ['UST (failed)', 'FRAX', 'FEI', 'AMPL'],
        'collateral': 'None or partial (endogenous)',
        'collateral_ratio': 0.0,  # 0% or variable
        'decentralization': 'Decentralized',
        'capital_efficiency': 'High (no collateral)',
        'risk_factors': ['Death spiral', 'Reflexivity', 'Bank run dynamics'],
        'typical_deviation': 0.02  # 2% typical deviation (higher volatility)
    }
}

def display_stablecoin_comparison():
    """Display a comparison table of stablecoin types."""
    print("=" * 90)
    print("STABLECOIN TYPE COMPARISON")
    print("=" * 90)
    
    for stype, info in stablecoin_types.items():
        print(f"\n{stype.upper()}")
        print("-" * 40)
        print(f"  Examples: {', '.join(info['examples'])}")
        print(f"  Collateral: {info['collateral']}")
        print(f"  Min Collateral Ratio: {info['collateral_ratio']*100:.0f}%")
        print(f"  Decentralization: {info['decentralization']}")
        print(f"  Capital Efficiency: {info['capital_efficiency']}")
        print(f"  Risk Factors: {', '.join(info['risk_factors'])}")
        print(f"  Typical Peg Deviation: +/- {info['typical_deviation']*100:.1f}%")
    
    print("\n" + "=" * 90)

display_stablecoin_comparison()

**Figure Interpretation:**
- **Left panel:** Shows the fundamental trade-off - you can't have perfect stability AND perfect decentralization
- **Middle panel:** Algorithmic stablecoins are 100% capital efficient (no collateral locked up), but...
- **Right panel:** ...they carry the highest risk (9/10 vs 2/10 for fiat-backed)

**Key insight:** Every stablecoin design chooses where to sit on these spectrums. There's no free lunch!

In [ ]:
# Visualize the trade-offs between stablecoin types
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

types = ['Fiat-Backed', 'Crypto-Backed', 'Algorithmic']
colors = ['#2ecc71', '#3498db', '#e74c3c']

# Plot 1: Stability vs Decentralization
stability = [0.95, 0.75, 0.3]  # Stability score
decentralization = [0.1, 0.8, 0.9]  # Decentralization score

ax1 = axes[0]
for i, t in enumerate(types):
    ax1.scatter(decentralization[i], stability[i], s=300, c=colors[i], label=t, alpha=0.7)
ax1.set_xlabel('Decentralization', fontsize=12)
ax1.set_ylabel('Stability', fontsize=12)
ax1.set_title('Stability vs Decentralization Trade-off', fontsize=12)
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Capital Efficiency
ax2 = axes[1]
capital_efficiency = [33, 66, 100]  # % capital efficiency
ax2.bar(types, capital_efficiency, color=colors, alpha=0.7)
ax2.set_ylabel('Capital Efficiency (%)', fontsize=12)
ax2.set_title('Capital Efficiency by Type', fontsize=12)
ax2.set_ylim(0, 120)
for i, v in enumerate(capital_efficiency):
    ax2.text(i, v + 3, f'{v}%', ha='center', fontsize=11)

# Plot 3: Risk Level
ax3 = axes[2]
risk_levels = [2, 5, 9]  # Risk score out of 10
ax3.barh(types, risk_levels, color=colors, alpha=0.7)
ax3.set_xlabel('Risk Level (1-10)', fontsize=12)
ax3.set_title('Risk Level by Type', fontsize=12)
ax3.set_xlim(0, 10)
for i, v in enumerate(risk_levels):
    ax3.text(v + 0.2, i, f'{v}/10', va='center', fontsize=11)

plt.tight_layout()
plt.suptitle('Stablecoin Design Trade-offs', fontsize=14, y=1.02)
plt.show()

print("\nKey Insight: There is no 'perfect' stablecoin design.")
print("Each type makes different trade-offs between stability, decentralization, and capital efficiency.")

## Section 3: Generate Synthetic Price Data

To analyze stablecoin behavior, we'll generate synthetic price data that mimics real-world patterns. Each stablecoin type has different volatility characteristics.

In [ ]:
def generate_stablecoin_prices(
    days: int = 365,
    base_price: float = 1.0,
    volatility: float = 0.001,
    mean_reversion_speed: float = 0.1,
    include_stress_events: bool = True,
    stress_probability: float = 0.02,
    stress_magnitude: float = 0.05
) -> np.ndarray:
    """
    Generate synthetic stablecoin price data using Ornstein-Uhlenbeck process
    with occasional stress events.
    
    The Ornstein-Uhlenbeck process models mean-reverting behavior:
    dP = theta * (mu - P) * dt + sigma * dW
    
    Args:
        days: Number of days to simulate
        base_price: Target peg price (typically $1)
        volatility: Daily volatility (standard deviation)
        mean_reversion_speed: How fast price returns to peg (0-1)
        include_stress_events: Whether to include random de-peg events
        stress_probability: Daily probability of a stress event
        stress_magnitude: Size of stress event deviation
    
    Returns:
        Array of daily prices
    """
    # Initialize price array
    prices = np.zeros(days)
    prices[0] = base_price
    
    for t in range(1, days):
        # Mean reversion component
        reversion = mean_reversion_speed * (base_price - prices[t-1])
        
        # Random shock
        shock = np.random.normal(0, volatility)
        
        # Stress event (occasional large deviation)
        if include_stress_events and np.random.random() < stress_probability:
            stress = np.random.choice([-1, 1]) * stress_magnitude * np.random.random()
        else:
            stress = 0
        
        # Update price
        prices[t] = prices[t-1] + reversion + shock + stress
        
        # Price floor (stablecoins can't go below 0)
        prices[t] = max(prices[t], 0.01)
    
    return prices

# Generate prices for each stablecoin type
np.random.seed(42)
days = 365

# Fiat-backed: Low volatility, fast mean reversion, rare stress
usdc_prices = generate_stablecoin_prices(
    days=days,
    volatility=0.0005,
    mean_reversion_speed=0.5,
    stress_probability=0.005,
    stress_magnitude=0.01
)

# Crypto-backed: Medium volatility, medium mean reversion
dai_prices = generate_stablecoin_prices(
    days=days,
    volatility=0.002,
    mean_reversion_speed=0.2,
    stress_probability=0.02,
    stress_magnitude=0.03
)

# Algorithmic: Higher volatility, slower mean reversion, more stress
algo_prices = generate_stablecoin_prices(
    days=days,
    volatility=0.005,
    mean_reversion_speed=0.1,
    stress_probability=0.05,
    stress_magnitude=0.08
)

print(f"Generated {days} days of synthetic price data for 3 stablecoin types.")
print(f"\nPrice statistics (should all be close to $1.00):")
print(f"  USDC (fiat-backed):  Mean=${usdc_prices.mean():.4f}, Std=${usdc_prices.std():.4f}")
print(f"  DAI (crypto-backed): Mean=${dai_prices.mean():.4f}, Std=${dai_prices.std():.4f}")
print(f"  ALGO (algorithmic):  Mean=${algo_prices.mean():.4f}, Std=${algo_prices.std():.4f}")

In [ ]:
# Plot the synthetic price data
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

dates = pd.date_range(start='2023-01-01', periods=days, freq='D')

# Plot 1: Full price history
ax1 = axes[0]
ax1.plot(dates, usdc_prices, label='USDC (Fiat-Backed)', color='#2ecc71', alpha=0.8)
ax1.plot(dates, dai_prices, label='DAI (Crypto-Backed)', color='#3498db', alpha=0.8)
ax1.plot(dates, algo_prices, label='Algorithmic', color='#e74c3c', alpha=0.8)
ax1.axhline(y=1.0, color='black', linestyle='--', alpha=0.5, label='$1.00 Peg')
ax1.axhline(y=0.99, color='gray', linestyle=':', alpha=0.3)
ax1.axhline(y=1.01, color='gray', linestyle=':', alpha=0.3)
ax1.fill_between(dates, 0.99, 1.01, alpha=0.1, color='green', label='+/- 1% Band')
ax1.set_xlabel('Date')
ax1.set_ylabel('Price (USD)')
ax1.set_title('Stablecoin Price Comparison Over 1 Year')
ax1.legend(loc='upper right')
ax1.set_ylim(0.90, 1.10)
ax1.grid(True, alpha=0.3)

# Plot 2: Deviation from peg
ax2 = axes[1]
ax2.plot(dates, (usdc_prices - 1) * 100, label='USDC', color='#2ecc71', alpha=0.8)
ax2.plot(dates, (dai_prices - 1) * 100, label='DAI', color='#3498db', alpha=0.8)
ax2.plot(dates, (algo_prices - 1) * 100, label='Algorithmic', color='#e74c3c', alpha=0.8)
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax2.axhline(y=1, color='gray', linestyle=':', alpha=0.3)
ax2.axhline(y=-1, color='gray', linestyle=':', alpha=0.3)
ax2.set_xlabel('Date')
ax2.set_ylabel('Deviation from Peg (%)')
ax2.set_title('Peg Deviation Over Time')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservation: Fiat-backed stablecoins (green) show the tightest peg,")
print("while algorithmic stablecoins (red) show the most volatility.")

**Figure Interpretation:**
- **Top panel:** Shows actual price over time. Notice how fiat-backed (green) stays closest to $1, while algorithmic (red) has wild swings
- **Bottom panel:** Deviation from peg - most interesting during stress events (the spikes). Fiat-backed stays within ±0.5%, algorithmic can hit ±5%

**What to look for:**
- Width of price band = volatility
- Frequency of deviations = stability
- Recovery speed = how fast price returns to $1 after a shock

## Section 4: Peg Deviation Analysis

Let's analyze how often and by how much each stablecoin type deviates from its $1 target. This is crucial for users who depend on stablecoins for payments or as a store of value.

In [ ]:
def analyze_peg_deviation(prices: np.ndarray, name: str, target: float = 1.0) -> Dict:
    """
    Comprehensive analysis of peg deviation.
    
    Args:
        prices: Array of price data
        name: Name of the stablecoin
        target: Target peg price
    
    Returns:
        Dictionary with deviation statistics
    """
    deviations = prices - target
    abs_deviations = np.abs(deviations)
    pct_deviations = deviations / target * 100
    
    stats = {
        'name': name,
        'mean_price': prices.mean(),
        'std_price': prices.std(),
        'min_price': prices.min(),
        'max_price': prices.max(),
        'mean_deviation': deviations.mean(),
        'std_deviation': deviations.std(),
        'max_deviation_above': deviations.max(),
        'max_deviation_below': deviations.min(),
        'pct_within_1_cent': (abs_deviations <= 0.01).sum() / len(prices) * 100,
        'pct_within_5_cents': (abs_deviations <= 0.05).sum() / len(prices) * 100,
        'days_below_99_cents': (prices < 0.99).sum(),
        'days_above_101_cents': (prices > 1.01).sum(),
        'max_consecutive_depeg_days': 0
    }
    
    # Calculate max consecutive days outside 1% band
    outside_band = (abs_deviations > 0.01).astype(int)
    consecutive = 0
    max_consecutive = 0
    for val in outside_band:
        if val == 1:
            consecutive += 1
            max_consecutive = max(max_consecutive, consecutive)
        else:
            consecutive = 0
    stats['max_consecutive_depeg_days'] = max_consecutive
    
    return stats

# Analyze each stablecoin
usdc_stats = analyze_peg_deviation(usdc_prices, 'USDC (Fiat-Backed)')
dai_stats = analyze_peg_deviation(dai_prices, 'DAI (Crypto-Backed)')
algo_stats = analyze_peg_deviation(algo_prices, 'Algorithmic')

def print_deviation_report(stats: Dict):
    """Print a formatted deviation report."""
    print(f"\n{'=' * 60}")
    print(f"{stats['name'].upper()}")
    print(f"{'=' * 60}")
    print(f"\nPrice Statistics:")
    print(f"  Mean Price:     ${stats['mean_price']:.4f}")
    print(f"  Std Deviation:  ${stats['std_price']:.4f}")
    print(f"  Price Range:    ${stats['min_price']:.4f} - ${stats['max_price']:.4f}")
    print(f"\nDeviation Analysis:")
    print(f"  Max Above Peg:  +${stats['max_deviation_above']:.4f} (+{stats['max_deviation_above']*100:.2f}%)")
    print(f"  Max Below Peg:  ${stats['max_deviation_below']:.4f} ({stats['max_deviation_below']*100:.2f}%)")
    print(f"\nStability Metrics:")
    print(f"  Days within 1 cent of peg: {stats['pct_within_1_cent']:.1f}%")
    print(f"  Days within 5 cents of peg: {stats['pct_within_5_cents']:.1f}%")
    print(f"  Days below $0.99: {stats['days_below_99_cents']} days")
    print(f"  Days above $1.01: {stats['days_above_101_cents']} days")
    print(f"  Max consecutive de-peg days: {stats['max_consecutive_depeg_days']} days")

# Print reports
print_deviation_report(usdc_stats)
print_deviation_report(dai_stats)
print_deviation_report(algo_stats)

In [ ]:
# Visualize deviation distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

stablecoins = [
    ('USDC', usdc_prices, '#2ecc71'),
    ('DAI', dai_prices, '#3498db'),
    ('Algorithmic', algo_prices, '#e74c3c')
]

for ax, (name, prices, color) in zip(axes, stablecoins):
    deviations = (prices - 1) * 100  # Convert to percentage
    
    # Histogram
    ax.hist(deviations, bins=50, color=color, alpha=0.7, edgecolor='black')
    ax.axvline(x=0, color='black', linestyle='--', linewidth=2)
    ax.axvline(x=1, color='gray', linestyle=':', alpha=0.5)
    ax.axvline(x=-1, color='gray', linestyle=':', alpha=0.5)
    
    # Add statistics
    ax.text(0.05, 0.95, f'Mean: {deviations.mean():.3f}%\nStd: {deviations.std():.3f}%',
            transform=ax.transAxes, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax.set_xlabel('Deviation from $1 (%)')
    ax.set_ylabel('Frequency (Days)')
    ax.set_title(f'{name} Price Distribution')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Distribution of Peg Deviations', fontsize=14, y=1.02)
plt.show()

print("\nKey Insight: The width of each distribution shows the stability of the peg.")
print("Narrower distributions = more stable stablecoin.")

In [ ]:
# Create comparison summary table
summary_data = {
    'Metric': [
        'Mean Price',
        'Std Deviation',
        'Max Deviation',
        '% Days Within 1 cent',
        '% Days Within 5 cents',
        'Days Outside 1% Band',
        'Max Consecutive De-peg'
    ],
    'USDC (Fiat)': [
        f"${usdc_stats['mean_price']:.4f}",
        f"${usdc_stats['std_price']:.4f}",
        f"{max(abs(usdc_stats['max_deviation_above']), abs(usdc_stats['max_deviation_below']))*100:.2f}%",
        f"{usdc_stats['pct_within_1_cent']:.1f}%",
        f"{usdc_stats['pct_within_5_cents']:.1f}%",
        f"{usdc_stats['days_below_99_cents'] + usdc_stats['days_above_101_cents']} days",
        f"{usdc_stats['max_consecutive_depeg_days']} days"
    ],
    'DAI (Crypto)': [
        f"${dai_stats['mean_price']:.4f}",
        f"${dai_stats['std_price']:.4f}",
        f"{max(abs(dai_stats['max_deviation_above']), abs(dai_stats['max_deviation_below']))*100:.2f}%",
        f"{dai_stats['pct_within_1_cent']:.1f}%",
        f"{dai_stats['pct_within_5_cents']:.1f}%",
        f"{dai_stats['days_below_99_cents'] + dai_stats['days_above_101_cents']} days",
        f"{dai_stats['max_consecutive_depeg_days']} days"
    ],
    'Algorithmic': [
        f"${algo_stats['mean_price']:.4f}",
        f"${algo_stats['std_price']:.4f}",
        f"{max(abs(algo_stats['max_deviation_above']), abs(algo_stats['max_deviation_below']))*100:.2f}%",
        f"{algo_stats['pct_within_1_cent']:.1f}%",
        f"{algo_stats['pct_within_5_cents']:.1f}%",
        f"{algo_stats['days_below_99_cents'] + algo_stats['days_above_101_cents']} days",
        f"{algo_stats['max_consecutive_depeg_days']} days"
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n" + "=" * 80)
print("STABLECOIN STABILITY COMPARISON")
print("=" * 80)
print(summary_df.to_string(index=False))
print("=" * 80)

**Figure Interpretation:**
- **Shape matters:** Narrow, tall distributions = stable. Wide, flat distributions = volatile
- **USDC (left):** Tightly clustered around 0% deviation - most days are exactly at peg
- **DAI (middle):** Wider spread, showing the impact of crypto collateral volatility
- **Algorithmic (right):** Long tails on both sides - can deviate significantly in either direction

**The "fat tails":** Those outlier days far from the peg are what you should worry about. They represent de-peg events that could trigger cascades.

## Section 5: Simulate Crypto-Backed Stablecoin (DAI-like)

Now let's build a simulation of a crypto-backed stablecoin system like MakerDAO's DAI. In this system:

1. Users deposit collateral (e.g., ETH) into a **Vault** (formerly CDP = Collateralized Debt Position)
2. They can mint stablecoins up to a certain **collateralization ratio** (e.g., 150%)
3. If collateral value drops, the position may be **liquidated**
4. The system maintains a peg through over-collateralization

### Understanding Vaults: The Pawn Shop Analogy

Think of a vault (or CDP - Collateralized Debt Position) like a digital pawn shop:

**Traditional Pawn Shop:**
- You bring in valuable jewelry worth $150
- The pawn shop gives you a $100 loan
- If you don't pay back the loan, they keep your jewelry and sell it

**Crypto Vault:**
- You deposit 1 ETH (worth $2000) as collateral
- The protocol lets you borrow up to $1333 in stablecoins (150% ratio)
- If ETH price drops too low, the vault is automatically liquidated (sold)
- The protocol sells your ETH to recover the loan

**Key difference:** In crypto, everything is automated by smart contracts. No human decides when to liquidate - the code does it instantly when your collateral value falls below the threshold.

### Why Over-Collateralize? The Risk Buffer Explained

Why require $150 of collateral for a $100 loan? Because crypto is volatile!

**Example:**
- You deposit $150 of ETH as collateral
- You borrow $100 in stablecoins
- ETH drops 20% → Your collateral is now worth $120
- Your loan is still covered! ($120 > $100)
- But if you only had 100% collateral ($100), a 1% drop would make you insolvent

**The 150% minimum ratio means:**
- ETH can drop up to 33% before your vault is at risk
- This buffer protects both you (time to add collateral) and the system (ensures loans are backed)
- The more volatile the asset, the higher the ratio should be

In [ ]:
@dataclass
class Vault:
    """
    A collateralized debt position (vault) for minting stablecoins.
    """
    owner: str
    collateral_amount: float  # Amount of ETH deposited
    debt: float  # Amount of stablecoin minted
    
    def collateralization_ratio(self, eth_price: float) -> float:
        """Calculate current collateralization ratio."""
        if self.debt == 0:
            return float('inf')
        collateral_value = self.collateral_amount * eth_price
        return collateral_value / self.debt
    
    def max_debt(self, eth_price: float, min_ratio: float = 1.5) -> float:
        """Calculate maximum debt allowed at current collateral level."""
        collateral_value = self.collateral_amount * eth_price
        return collateral_value / min_ratio
    
    def is_liquidatable(self, eth_price: float, liquidation_ratio: float = 1.5) -> bool:
        """Check if vault can be liquidated."""
        return self.collateralization_ratio(eth_price) < liquidation_ratio


class CryptoBackedStablecoin:
    """
    Simulation of a crypto-backed stablecoin system (like MakerDAO).
    """
    
    def __init__(
        self,
        name: str = "SimDAI",
        min_collateral_ratio: float = 1.5,  # 150%
        liquidation_ratio: float = 1.5,     # 150%
        liquidation_penalty: float = 0.13,  # 13% penalty
        stability_fee: float = 0.02         # 2% annual
    ):
        self.name = name
        self.min_collateral_ratio = min_collateral_ratio
        self.liquidation_ratio = liquidation_ratio
        self.liquidation_penalty = liquidation_penalty
        self.stability_fee = stability_fee
        
        self.vaults: Dict[str, Vault] = {}
        self.total_supply = 0.0
        self.total_collateral = 0.0
        self.liquidation_history: List[Dict] = []
        
    def open_vault(self, owner: str, collateral: float, debt: float, eth_price: float) -> bool:
        """
        Open a new vault with collateral and mint stablecoins.
        
        Returns True if successful, False if under-collateralized.
        """
        vault = Vault(owner=owner, collateral_amount=collateral, debt=0)
        
        # Check if requested debt is within limits
        max_allowed = vault.max_debt(eth_price, self.min_collateral_ratio)
        
        if debt > max_allowed:
            print(f"Rejected: Requested debt ${debt:.2f} exceeds max ${max_allowed:.2f}")
            return False
        
        vault.debt = debt
        self.vaults[owner] = vault
        self.total_supply += debt
        self.total_collateral += collateral
        
        ratio = vault.collateralization_ratio(eth_price)
        print(f"Vault opened for {owner}:")
        print(f"  Collateral: {collateral:.2f} ETH (${collateral * eth_price:.2f})")
        print(f"  Debt: {debt:.2f} {self.name}")
        print(f"  Collateralization Ratio: {ratio:.1%}")
        
        return True
    
    def check_liquidations(self, eth_price: float, timestamp: int = 0) -> List[str]:
        """
        Check all vaults for liquidation conditions and liquidate if necessary.
        
        Returns list of liquidated vault owners.
        """
        liquidated = []
        
        for owner, vault in list(self.vaults.items()):
            if vault.is_liquidatable(eth_price, self.liquidation_ratio):
                ratio = vault.collateralization_ratio(eth_price)
                collateral_value = vault.collateral_amount * eth_price
                
                # Record liquidation
                self.liquidation_history.append({
                    'timestamp': timestamp,
                    'owner': owner,
                    'collateral': vault.collateral_amount,
                    'debt': vault.debt,
                    'eth_price': eth_price,
                    'ratio': ratio,
                    'penalty': vault.debt * self.liquidation_penalty
                })
                
                # Update totals
                self.total_supply -= vault.debt
                self.total_collateral -= vault.collateral_amount
                
                # Remove vault
                del self.vaults[owner]
                liquidated.append(owner)
        
        return liquidated
    
    def system_stats(self, eth_price: float) -> Dict:
        """Get current system statistics."""
        if self.total_supply == 0:
            system_ratio = float('inf')
        else:
            system_ratio = (self.total_collateral * eth_price) / self.total_supply
        
        return {
            'total_supply': self.total_supply,
            'total_collateral_eth': self.total_collateral,
            'total_collateral_usd': self.total_collateral * eth_price,
            'system_collateral_ratio': system_ratio,
            'num_vaults': len(self.vaults),
            'total_liquidations': len(self.liquidation_history)
        }

# Create stablecoin system
simdai = CryptoBackedStablecoin(
    name="SimDAI",
    min_collateral_ratio=1.5,
    liquidation_ratio=1.5,
    liquidation_penalty=0.13
)

print("\nCrypto-Backed Stablecoin System Created")
print("=" * 50)
print(f"Name: {simdai.name}")
print(f"Minimum Collateral Ratio: {simdai.min_collateral_ratio:.0%}")
print(f"Liquidation Ratio: {simdai.liquidation_ratio:.0%}")
print(f"Liquidation Penalty: {simdai.liquidation_penalty:.0%}")

In [ ]:
# Open some example vaults at ETH = $2000
eth_price = 2000.0

print(f"\nOpening vaults at ETH price: ${eth_price}")
print("=" * 50)

# Alice: Conservative (200% collateralization)
print("\n--- Alice (Conservative) ---")
simdai.open_vault('Alice', collateral=10.0, debt=10000.0, eth_price=eth_price)

# Bob: Moderate (175% collateralization)
print("\n--- Bob (Moderate) ---")
simdai.open_vault('Bob', collateral=8.75, debt=10000.0, eth_price=eth_price)

# Charlie: Aggressive (155% collateralization - close to liquidation)
print("\n--- Charlie (Aggressive) ---")
simdai.open_vault('Charlie', collateral=7.75, debt=10000.0, eth_price=eth_price)

# Dave: Too aggressive (will be rejected)
print("\n--- Dave (Too Aggressive) ---")
simdai.open_vault('Dave', collateral=5.0, debt=10000.0, eth_price=eth_price)

# System stats
stats = simdai.system_stats(eth_price)
print(f"\n{'=' * 50}")
print("SYSTEM STATISTICS")
print(f"{'=' * 50}")
print(f"Total {simdai.name} Supply: ${stats['total_supply']:,.2f}")
print(f"Total Collateral: {stats['total_collateral_eth']:.2f} ETH (${stats['total_collateral_usd']:,.2f})")
print(f"System Collateral Ratio: {stats['system_collateral_ratio']:.1%}")
print(f"Active Vaults: {stats['num_vaults']}")

## Section 6: Collateralization Ratio Dynamics

Let's simulate how collateralization ratios change as ETH price fluctuates, and when liquidations are triggered.

In [ ]:
def simulate_price_impact(system: CryptoBackedStablecoin, eth_prices: List[float]):
    """
    Simulate the impact of ETH price changes on the stablecoin system.
    """
    results = []
    
    for i, price in enumerate(eth_prices):
        # Check for liquidations
        liquidated = system.check_liquidations(price, timestamp=i)
        
        # Get system stats
        stats = system.system_stats(price)
        
        # Get individual vault ratios
        vault_ratios = {}
        for owner, vault in system.vaults.items():
            vault_ratios[owner] = vault.collateralization_ratio(price)
        
        results.append({
            'timestamp': i,
            'eth_price': price,
            'liquidated': liquidated,
            'system_ratio': stats['system_collateral_ratio'],
            'total_supply': stats['total_supply'],
            'num_vaults': stats['num_vaults'],
            'vault_ratios': vault_ratios.copy()
        })
    
    return results

# Reset the system for simulation
simdai = CryptoBackedStablecoin(name="SimDAI")
initial_eth_price = 2000.0

# Open vaults silently
simdai.vaults['Alice'] = Vault('Alice', 10.0, 10000.0)  # 200%
simdai.vaults['Bob'] = Vault('Bob', 8.75, 10000.0)      # 175%
simdai.vaults['Charlie'] = Vault('Charlie', 7.75, 10000.0)  # 155%
simdai.total_supply = 30000.0
simdai.total_collateral = 26.5

# Simulate ETH price decline from $2000 to $1200
eth_price_scenario = np.linspace(2000, 1200, 50)

# Run simulation
simulation_results = simulate_price_impact(simdai, eth_price_scenario)

print("\nSimulation: ETH Price Decline from $2000 to $1200")
print("=" * 60)

In [ ]:
# Visualize the simulation results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

timestamps = [r['timestamp'] for r in simulation_results]
eth_prices = [r['eth_price'] for r in simulation_results]

# Plot 1: ETH Price
ax1 = axes[0, 0]
ax1.plot(timestamps, eth_prices, 'b-', linewidth=2)
ax1.set_xlabel('Time Step')
ax1.set_ylabel('ETH Price ($)')
ax1.set_title('ETH Price Scenario')
ax1.grid(True, alpha=0.3)

# Highlight liquidation points
for r in simulation_results:
    if r['liquidated']:
        ax1.axvline(x=r['timestamp'], color='red', linestyle='--', alpha=0.5)

# Plot 2: Individual Vault Collateralization Ratios
ax2 = axes[0, 1]

# Track vault ratios over time
alice_ratios = []
bob_ratios = []
charlie_ratios = []

for r in simulation_results:
    alice_ratios.append(r['vault_ratios'].get('Alice', np.nan))
    bob_ratios.append(r['vault_ratios'].get('Bob', np.nan))
    charlie_ratios.append(r['vault_ratios'].get('Charlie', np.nan))

ax2.plot(timestamps, [r * 100 if not np.isnan(r) else np.nan for r in alice_ratios], 
         'g-', linewidth=2, label='Alice (Conservative)')
ax2.plot(timestamps, [r * 100 if not np.isnan(r) else np.nan for r in bob_ratios], 
         'b-', linewidth=2, label='Bob (Moderate)')
ax2.plot(timestamps, [r * 100 if not np.isnan(r) else np.nan for r in charlie_ratios], 
         'r-', linewidth=2, label='Charlie (Aggressive)')
ax2.axhline(y=150, color='red', linestyle='--', alpha=0.7, label='Liquidation Threshold (150%)')
ax2.set_xlabel('Time Step')
ax2.set_ylabel('Collateralization Ratio (%)')
ax2.set_title('Individual Vault Collateralization')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(100, 250)

# Plot 3: System-wide Collateralization
ax3 = axes[1, 0]
system_ratios = [r['system_ratio'] * 100 for r in simulation_results]
ax3.plot(timestamps, system_ratios, 'purple', linewidth=2)
ax3.axhline(y=150, color='red', linestyle='--', alpha=0.7, label='Minimum Ratio')
ax3.set_xlabel('Time Step')
ax3.set_ylabel('System Collateralization Ratio (%)')
ax3.set_title('System-wide Collateralization')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Total Supply and Active Vaults
ax4 = axes[1, 1]
total_supplies = [r['total_supply'] for r in simulation_results]
num_vaults = [r['num_vaults'] for r in simulation_results]

ax4_twin = ax4.twinx()
line1, = ax4.plot(timestamps, total_supplies, 'b-', linewidth=2, label='Total Supply')
line2, = ax4_twin.plot(timestamps, num_vaults, 'g-', linewidth=2, label='Active Vaults')
ax4.set_xlabel('Time Step')
ax4.set_ylabel('Total Supply ($)', color='blue')
ax4_twin.set_ylabel('Active Vaults', color='green')
ax4.set_title('System Supply and Vaults')
ax4.legend(handles=[line1, line2], loc='upper right')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Crypto-Backed Stablecoin System Under ETH Price Decline', fontsize=14, y=1.02)
plt.show()

# Print liquidation events
print("\nLiquidation Events:")
print("-" * 60)
for event in simdai.liquidation_history:
    print(f"Step {event['timestamp']}: {event['owner']} liquidated")
    print(f"  ETH Price: ${event['eth_price']:.2f}")
    print(f"  Collateral: {event['collateral']:.2f} ETH")
    print(f"  Debt: ${event['debt']:.2f}")
    print(f"  Ratio at liquidation: {event['ratio']:.1%}")
    print(f"  Penalty: ${event['penalty']:.2f}")
    print()

## Section 7: Liquidation Cascade Simulation

A **liquidation cascade** occurs when liquidations trigger further price drops, causing more liquidations in a feedback loop. This is one of the biggest risks in DeFi.

The mechanism:
1. ETH price drops -> some vaults get liquidated
2. Liquidated collateral is sold on the market
3. Large sell orders push ETH price down further
4. More vaults become undercollateralized
5. Repeat until market stabilizes

**Figure Interpretation:**
- **Top left:** ETH price falling from $2000 to $1200 (40% drop). Red dashed lines = liquidation events
- **Top right:** Individual vault health deteriorating. Charlie (aggressive) gets liquidated first, then Bob. Alice (conservative) survives
- **Bottom left:** System-wide health remains above minimum because riskiest positions are liquidated
- **Bottom right:** Total supply drops as vaults are closed, remaining vaults decrease

**Key lesson:** Conservative positioning (200% ratio) survives 40% crashes. Aggressive positioning (155%) gets wiped out in moderate downturns.

In [ ]:
def simulate_liquidation_cascade(
    num_vaults: int = 100,
    initial_eth_price: float = 2000.0,
    price_shock: float = 0.20,  # 20% initial drop
    market_depth: float = 0.001,  # Price impact per ETH sold (0.1%)
    iterations: int = 50
) -> Dict:
    """
    Simulate a liquidation cascade scenario.
    
    Args:
        num_vaults: Number of vaults in the system
        initial_eth_price: Starting ETH price
        price_shock: Initial price drop percentage
        market_depth: Price impact coefficient (price drop per ETH sold)
        iterations: Number of iterations to simulate
    
    Returns:
        Dictionary with simulation results
    """
    # Create system with many vaults at different collateral ratios
    system = CryptoBackedStablecoin(name="SimDAI")
    
    # Generate vaults with varying collateralization (150% - 250%)
    np.random.seed(42)
    collateral_ratios = np.random.uniform(1.5, 2.5, num_vaults)
    
    for i in range(num_vaults):
        debt = 10000.0  # Each vault has $10k debt
        collateral_value = debt * collateral_ratios[i]
        collateral_eth = collateral_value / initial_eth_price
        
        vault = Vault(f"Vault_{i}", collateral_eth, debt)
        system.vaults[f"Vault_{i}"] = vault
        system.total_supply += debt
        system.total_collateral += collateral_eth
    
    # Track simulation
    history = {
        'eth_price': [initial_eth_price],
        'total_supply': [system.total_supply],
        'total_collateral': [system.total_collateral],
        'num_vaults': [len(system.vaults)],
        'liquidations_per_step': [0],
        'eth_sold': [0],
        'system_ratio': [(system.total_collateral * initial_eth_price) / system.total_supply]
    }
    
    # Apply initial price shock
    current_price = initial_eth_price * (1 - price_shock)
    
    for iteration in range(iterations):
        # Check for liquidations
        liquidations_before = len(system.liquidation_history)
        liquidated = system.check_liquidations(current_price, timestamp=iteration)
        
        # Calculate ETH sold in liquidations
        eth_sold = sum(
            liq['collateral'] 
            for liq in system.liquidation_history[liquidations_before:]
        )
        
        # Price impact from liquidations
        price_impact = eth_sold * market_depth * current_price
        current_price = max(current_price - price_impact, 100)  # Floor at $100
        
        # Record state
        history['eth_price'].append(current_price)
        history['total_supply'].append(system.total_supply)
        history['total_collateral'].append(system.total_collateral)
        history['num_vaults'].append(len(system.vaults))
        history['liquidations_per_step'].append(len(liquidated))
        history['eth_sold'].append(eth_sold)
        
        if system.total_supply > 0:
            history['system_ratio'].append(
                (system.total_collateral * current_price) / system.total_supply
            )
        else:
            history['system_ratio'].append(0)
        
        # Check if cascade has stopped
        if len(liquidated) == 0:
            break
    
    return {
        'history': history,
        'system': system,
        'final_price': current_price,
        'total_liquidations': len(system.liquidation_history)
    }

# Run cascade simulation
print("Simulating Liquidation Cascade...")
print("=" * 60)
print(f"Initial Conditions:")
print(f"  - 100 vaults with $10,000 debt each")
print(f"  - Collateralization ratios: 150% - 250%")
print(f"  - Initial ETH Price: $2,000")
print(f"  - Price Shock: -20% (drops to $1,600)")
print(f"  - Market Depth: 0.1% per ETH sold")

cascade_result = simulate_liquidation_cascade(
    num_vaults=100,
    initial_eth_price=2000.0,
    price_shock=0.20,
    market_depth=0.001,
    iterations=50
)

print(f"\nResults:")
print(f"  - Final ETH Price: ${cascade_result['final_price']:.2f}")
print(f"  - Price dropped additional: {(1 - cascade_result['final_price']/1600)*100:.1f}% from cascade")
print(f"  - Total Liquidations: {cascade_result['total_liquidations']}")
print(f"  - Remaining Vaults: {len(cascade_result['system'].vaults)}")

In [ ]:
# Visualize the liquidation cascade
history = cascade_result['history']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

steps = range(len(history['eth_price']))

# Plot 1: ETH Price During Cascade
ax1 = axes[0, 0]
ax1.plot(steps, history['eth_price'], 'b-', linewidth=2)
ax1.axhline(y=2000, color='green', linestyle='--', alpha=0.5, label='Initial Price')
ax1.axhline(y=1600, color='orange', linestyle='--', alpha=0.5, label='After Shock')
ax1.fill_between(steps, history['eth_price'], 1600, 
                  where=[p < 1600 for p in history['eth_price']], 
                  color='red', alpha=0.3, label='Cascade Impact')
ax1.set_xlabel('Iteration')
ax1.set_ylabel('ETH Price ($)')
ax1.set_title('ETH Price During Liquidation Cascade')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Liquidations Per Step
ax2 = axes[0, 1]
ax2.bar(steps, history['liquidations_per_step'], color='red', alpha=0.7)
ax2.set_xlabel('Iteration')
ax2.set_ylabel('Liquidations')
ax2.set_title('Liquidations Per Iteration')
ax2.grid(True, alpha=0.3)

# Plot 3: Remaining Vaults
ax3 = axes[1, 0]
ax3.plot(steps, history['num_vaults'], 'g-', linewidth=2)
ax3.fill_between(steps, history['num_vaults'], 0, alpha=0.3, color='green')
ax3.set_xlabel('Iteration')
ax3.set_ylabel('Active Vaults')
ax3.set_title('Surviving Vaults Over Time')
ax3.grid(True, alpha=0.3)

# Plot 4: System Collateralization Ratio
ax4 = axes[1, 1]
ratios_pct = [r * 100 for r in history['system_ratio']]
ax4.plot(steps, ratios_pct, 'purple', linewidth=2)
ax4.axhline(y=150, color='red', linestyle='--', alpha=0.7, label='Min Ratio (150%)')
ax4.set_xlabel('Iteration')
ax4.set_ylabel('System Collateralization (%)')
ax4.set_title('System-wide Collateralization During Cascade')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Liquidation Cascade Dynamics', fontsize=14, y=1.02)
plt.show()

print("\nKey Observations:")
print("1. The initial 20% price shock triggers a wave of liquidations")
print("2. Liquidated collateral being sold pushes prices lower")
print("3. Lower prices trigger more liquidations (cascade effect)")
print("4. The cascade continues until no more positions are liquidatable")
print("5. Final price is lower than what the initial shock alone would cause")

In [ ]:
# Compare different market depth scenarios
market_depths = [0.0005, 0.001, 0.002, 0.005]
depth_results = []

print("\nComparing Different Market Depth Scenarios")
print("=" * 60)
print("Market depth = price impact per ETH sold")
print("Higher depth = less liquid market = more severe cascades\n")

for depth in market_depths:
    result = simulate_liquidation_cascade(
        num_vaults=100,
        initial_eth_price=2000.0,
        price_shock=0.20,
        market_depth=depth,
        iterations=50
    )
    depth_results.append(result)
    
    price_drop = (1 - result['final_price'] / 2000) * 100
    cascade_extra = (1 - result['final_price'] / 1600) * 100
    
    print(f"Depth {depth*100:.2f}%/ETH:")
    print(f"  Final Price: ${result['final_price']:.2f} ({price_drop:.1f}% total drop)")
    print(f"  Cascade Added: {cascade_extra:.1f}% additional drop")
    print(f"  Liquidations: {result['total_liquidations']}/{100} vaults")
    print()

# Plot comparison
fig, ax = plt.subplots(figsize=(12, 6))

colors = plt.cm.Reds(np.linspace(0.3, 0.9, len(market_depths)))

for i, (depth, result) in enumerate(zip(market_depths, depth_results)):
    ax.plot(range(len(result['history']['eth_price'])), 
            result['history']['eth_price'],
            color=colors[i], linewidth=2,
            label=f"Depth: {depth*100:.2f}%/ETH")

ax.axhline(y=2000, color='gray', linestyle='--', alpha=0.5)
ax.axhline(y=1600, color='gray', linestyle=':', alpha=0.5)
ax.set_xlabel('Iteration', fontsize=12)
ax.set_ylabel('ETH Price ($)', fontsize=12)
ax.set_title('Impact of Market Depth on Liquidation Cascades', fontsize=14)
ax.legend(title='Market Depth')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nConclusion: Lower liquidity (higher market depth) leads to more severe cascades.")
print("This is why DeFi protocols are most vulnerable during market stress when liquidity dries up.")

### Understanding the Terra Death Spiral: A Visual Feedback Loop

Before we simulate the collapse, let's understand the **doom loop** that destroyed Terra/UST:

```
┌─────────────────────────────────────────────────────────────────┐
│                    THE TERRA DEATH SPIRAL                        │
└─────────────────────────────────────────────────────────────────┘

Step 1: Initial Trigger
   UST loses peg → Drops to $0.98
              ↓
Step 2: Arbitrage Response
   Users burn UST to get $1 worth of LUNA → Massive LUNA minting
              ↓
Step 3: LUNA Supply Shock
   LUNA supply explodes → LUNA price crashes
              ↓
Step 4: Confidence Crisis
   Falling LUNA price → Less confidence in UST backing
              ↓
Step 5: Acceleration
   UST drops to $0.90 → Even MORE arbitrage redemptions
              ↓
Step 6: Cascade
   More LUNA minted → LUNA crashes harder → UST crashes more
              ↓
Step 7: Total Collapse
   Both UST and LUNA approach $0
```

**Why couldn't it recover?**

1. **Reflexivity:** The mechanism to restore the peg (burning UST for LUNA) relied on LUNA having value
2. **Confidence dependency:** Once people lost faith, arbitrage couldn't work
3. **No floor:** Unlike crypto-backed stablecoins with real collateral, there was no hard backstop
4. **Speed:** The collapse happened in 72 hours - too fast for any intervention

**The feedback loop is key:** Each step makes the next step worse, accelerating the collapse.

**Figure Interpretation:**
- **Top left:** Price doesn't stop at the initial shock level ($1600). The cascade pushes it lower as liquidations create selling pressure
- **Top right:** Liquidations happen in waves - first wave triggers second wave, etc.
- **Bottom left:** Surviving vaults decrease in steps as each wave hits
- **Bottom right:** System collateralization improves (paradoxically) because weak positions are eliminated

**The cascade effect:** The additional price drop BEYOND the initial shock is pure cascade. In this simulation, a 20% shock became a ~35% crash due to feedback effects.

## Section 8: Historical Case Study - Terra/UST Collapse

In May 2022, the Terra ecosystem collapsed in one of the largest failures in crypto history. UST, an algorithmic stablecoin, lost its peg and crashed to near zero, destroying $40+ billion in value.

### How Terra/UST Worked

1. **Algorithmic Mechanism**: UST maintained its peg through a mint/burn mechanism with LUNA
2. **Arbitrage**: If UST < $1, users could burn UST for $1 worth of LUNA (profit)
3. **If UST > $1**, users could mint UST by burning $1 worth of LUNA
4. **Anchor Protocol**: Offered 19.5% APY on UST deposits (unsustainable yield)

### The Death Spiral

1. Large withdrawals from Anchor caused UST selling pressure
2. UST started depegging (falling below $1)
3. Arbitrageurs burned UST for LUNA, massively increasing LUNA supply
4. More LUNA supply -> LUNA price crashes
5. Lower LUNA price -> harder to absorb UST selling
6. Panic selling -> both UST and LUNA collapse to near zero

In [ ]:
def simulate_algorithmic_stablecoin_collapse(
    initial_ust_supply: float = 18e9,  # $18 billion
    initial_luna_supply: float = 350e6,  # 350 million LUNA
    initial_luna_price: float = 80.0,  # $80 per LUNA
    initial_ust_price: float = 1.0,
    daily_redemption_rate: float = 0.05,  # 5% of UST redeemed per day
    confidence_decay: float = 0.1,  # How fast confidence erodes
    days: int = 14
) -> Dict:
    """
    Simulate an algorithmic stablecoin death spiral.
    
    This models the reflexive dynamics that caused Terra/UST to collapse.
    """
    history = {
        'day': [],
        'ust_price': [],
        'luna_price': [],
        'ust_supply': [],
        'luna_supply': [],
        'luna_market_cap': [],
        'ust_market_cap': [],
        'confidence': []
    }
    
    ust_supply = initial_ust_supply
    luna_supply = initial_luna_supply
    luna_price = initial_luna_price
    ust_price = initial_ust_price
    confidence = 1.0  # Market confidence (0-1)
    
    for day in range(days):
        # Record state
        history['day'].append(day)
        history['ust_price'].append(ust_price)
        history['luna_price'].append(luna_price)
        history['ust_supply'].append(ust_supply)
        history['luna_supply'].append(luna_supply)
        history['luna_market_cap'].append(luna_supply * luna_price)
        history['ust_market_cap'].append(ust_supply * ust_price)
        history['confidence'].append(confidence)
        
        # Calculate redemption amount based on de-peg severity
        depeg_severity = max(0, 1 - ust_price)  # How far below $1
        panic_factor = 1 + depeg_severity * 5  # Panic increases redemptions
        
        redemption_rate = daily_redemption_rate * panic_factor * (1 - confidence)
        redemption_rate = min(redemption_rate, 0.5)  # Cap at 50% per day
        
        ust_redeemed = ust_supply * redemption_rate
        
        if ust_redeemed > 0 and luna_price > 0.001:
            # Burn UST, mint LUNA (at theoretical $1 redemption value)
            luna_minted = ust_redeemed / luna_price
            
            ust_supply -= ust_redeemed
            luna_supply += luna_minted
            
            # LUNA price drops due to massive supply increase
            supply_increase = luna_minted / (luna_supply - luna_minted)
            luna_price *= (1 - supply_increase * 0.8)  # Price impact
            luna_price = max(luna_price, 0.0001)  # Floor
            
            # UST price reflects reduced backing
            luna_market_cap = luna_supply * luna_price
            backing_ratio = luna_market_cap / ust_supply if ust_supply > 0 else 0
            
            # UST price depends on confidence and backing
            ust_price = min(1.0, backing_ratio * confidence)
            ust_price = max(ust_price, 0.001)
        
        # Confidence erodes based on depeg
        confidence *= (1 - confidence_decay * depeg_severity)
        confidence = max(confidence, 0.01)
    
    return history

# Run Terra collapse simulation
print("Simulating Algorithmic Stablecoin Collapse (Terra/UST Style)")
print("=" * 60)

collapse_history = simulate_algorithmic_stablecoin_collapse(
    initial_ust_supply=18e9,
    initial_luna_supply=350e6,
    initial_luna_price=80.0,
    daily_redemption_rate=0.05,
    confidence_decay=0.15,
    days=14
)

**Figure Interpretation:**
- **Red line (highest depth):** Low liquidity market - same liquidations cause massive price impact
- **Light red → Pink:** Progressively deeper markets absorb selling pressure better
- **All lines start at $2000, drop to $1600:** Same initial shock, different outcomes

**Real-world application:** This is why DeFi protocols are most vulnerable during market stress - exactly when liquidity dries up and market depth shrinks. The death spiral accelerates when you need resilience most.

In [ ]:
# Visualize the collapse
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

days = collapse_history['day']

# Plot 1: UST Price (De-peg)
ax1 = axes[0, 0]
ax1.plot(days, collapse_history['ust_price'], 'b-', linewidth=2, marker='o')
ax1.axhline(y=1.0, color='green', linestyle='--', alpha=0.5, label='$1 Peg')
ax1.fill_between(days, collapse_history['ust_price'], 1.0, 
                  where=[p < 1 for p in collapse_history['ust_price']],
                  color='red', alpha=0.3)
ax1.set_xlabel('Day')
ax1.set_ylabel('UST Price ($)')
ax1.set_title('UST Price Collapse')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1.1)

# Plot 2: LUNA Price
ax2 = axes[0, 1]
ax2.semilogy(days, collapse_history['luna_price'], 'r-', linewidth=2, marker='o')
ax2.set_xlabel('Day')
ax2.set_ylabel('LUNA Price ($) [Log Scale]')
ax2.set_title('LUNA Price Collapse')
ax2.grid(True, alpha=0.3)

# Plot 3: Supply Changes
ax3 = axes[1, 0]
ax3_twin = ax3.twinx()

ust_supply_b = [s / 1e9 for s in collapse_history['ust_supply']]
luna_supply_t = [s / 1e12 for s in collapse_history['luna_supply']]  # Trillions

line1, = ax3.plot(days, ust_supply_b, 'b-', linewidth=2, label='UST Supply')
line2, = ax3_twin.semilogy(days, luna_supply_t, 'r-', linewidth=2, label='LUNA Supply')

ax3.set_xlabel('Day')
ax3.set_ylabel('UST Supply (Billions)', color='blue')
ax3_twin.set_ylabel('LUNA Supply (Trillions) [Log]', color='red')
ax3.set_title('Supply Dynamics')
ax3.legend(handles=[line1, line2], loc='right')
ax3.grid(True, alpha=0.3)

# Plot 4: Market Confidence
ax4 = axes[1, 1]
ax4.plot(days, [c * 100 for c in collapse_history['confidence']], 'purple', linewidth=2, marker='o')
ax4.fill_between(days, [c * 100 for c in collapse_history['confidence']], 0, alpha=0.3, color='purple')
ax4.set_xlabel('Day')
ax4.set_ylabel('Market Confidence (%)')
ax4.set_title('Market Confidence Erosion')
ax4.set_ylim(0, 100)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Algorithmic Stablecoin Death Spiral Simulation', fontsize=14, y=1.02)
plt.show()

# Print final state
print("\nCollapse Summary:")
print("-" * 60)
print(f"Initial State:")
print(f"  UST: ${collapse_history['ust_supply'][0]/1e9:.1f}B at ${collapse_history['ust_price'][0]:.2f}")
print(f"  LUNA: {collapse_history['luna_supply'][0]/1e6:.0f}M at ${collapse_history['luna_price'][0]:.2f}")
print(f"  Total Market Cap: ${(collapse_history['ust_market_cap'][0] + collapse_history['luna_market_cap'][0])/1e9:.1f}B")
print(f"\nFinal State (Day {len(days)-1}):")
print(f"  UST: ${collapse_history['ust_supply'][-1]/1e9:.1f}B at ${collapse_history['ust_price'][-1]:.4f}")
print(f"  LUNA: {collapse_history['luna_supply'][-1]/1e12:.1f}T at ${collapse_history['luna_price'][-1]:.6f}")
print(f"  Total Market Cap: ${(collapse_history['ust_market_cap'][-1] + collapse_history['luna_market_cap'][-1])/1e9:.2f}B")
print(f"\nValue Destroyed: ${(collapse_history['ust_market_cap'][0] + collapse_history['luna_market_cap'][0] - collapse_history['ust_market_cap'][-1] - collapse_history['luna_market_cap'][-1])/1e9:.1f}B")

In [ ]:
# Key lessons from Terra collapse
print("\n" + "=" * 80)
print("KEY LESSONS FROM THE TERRA/UST COLLAPSE")
print("=" * 80)

lessons = [
    ("1. Reflexivity is Dangerous",
     "When the mechanism to maintain the peg relies on a token whose value depends on " +
     "confidence in the peg, you get a circular dependency that can unwind catastrophically."),
    
    ("2. 'Algorithmic' Doesn't Mean 'Safe'",
     "Code running automatically doesn't eliminate economic risk. The algorithm worked exactly " +
     "as designed - it just had a fatal flaw in its economic design."),
    
    ("3. Unsustainable Yields Are Red Flags",
     "Anchor's 19.5% APY on a stablecoin was a warning sign. Ask: where does yield come from? " +
     "If you can't answer, you might be the yield."),
    
    ("4. Market Depth Matters in Crises",
     "The same mechanism that worked at small scale failed spectacularly at large scale because " +
     "markets couldn't absorb the selling pressure."),
    
    ("5. Confidence is Fragile",
     "Once confidence breaks, it's almost impossible to restore. The speed of collapse " +
     "(~72 hours for most of the damage) gave no time for recovery."),
    
    ("6. Over-Collateralization Has Value",
     "Crypto-backed stablecoins like DAI survived because they have real collateral backing. " +
     "Capital inefficiency is the price of robustness.")
]

for title, explanation in lessons:
    print(f"\n{title}")
    print(f"   {explanation}")

print("\n" + "=" * 80)

## Section 9: Challenge Exercises

Test your understanding with these exercises:

### Challenge 1: Optimal Collateralization Strategy

Given historical ETH volatility, what is the optimal collateralization ratio that balances capital efficiency with liquidation risk?

**Figure Interpretation:**
- **Top left:** UST price collapse - notice it doesn't gradually decline, it cliff-dives once confidence breaks
- **Top right:** LUNA price on log scale (exponential decay) - went from $80 to fractions of a cent
- **Bottom left:** The deadly combination - UST supply shrinking while LUNA supply EXPLODES (note the trillion scale)
- **Bottom right:** Market confidence (purple) - once it breaks, it never recovers

**The speed is terrifying:** Most damage in first 3-4 days. This gave holders almost no time to exit. Compare this to slow price declines where you can react.

In [ ]:
def analyze_collateralization_strategies(
    ratios: List[float] = [1.5, 1.75, 2.0, 2.5, 3.0],
    simulations: int = 100,
    days: int = 365,
    initial_eth_price: float = 2000.0,
    eth_volatility: float = 0.05  # 5% daily volatility
) -> pd.DataFrame:
    """
    Analyze different collateralization strategies through Monte Carlo simulation.
    
    TODO: Complete this function to:
    1. Simulate ETH price paths with geometric Brownian motion
    2. For each path, check if each collateralization ratio would survive
    3. Calculate metrics: survival rate, capital efficiency, risk-adjusted return
    """
    results = []
    
    for ratio in ratios:
        survivals = 0
        min_ratios = []
        
        for sim in range(simulations):
            # Generate ETH price path (geometric Brownian motion)
            # TODO: Implement GBM price simulation
            returns = np.random.normal(0, eth_volatility, days)
            price_path = initial_eth_price * np.exp(np.cumsum(returns))
            
            # Calculate collateral ratio over time
            # Assume: 1 ETH collateral, debt = initial_price / ratio
            debt = initial_eth_price / ratio
            collateral_ratios = price_path / debt
            
            # Check if survived (never below 150%)
            min_ratio = collateral_ratios.min()
            min_ratios.append(min_ratio)
            
            if min_ratio >= 1.5:
                survivals += 1
        
        survival_rate = survivals / simulations
        avg_min_ratio = np.mean(min_ratios)
        capital_efficiency = 1 / ratio  # Higher ratio = lower efficiency
        
        results.append({
            'Initial Ratio': f"{ratio:.0%}",
            'Survival Rate': f"{survival_rate:.1%}",
            'Avg Min Ratio': f"{avg_min_ratio:.1%}",
            'Capital Efficiency': f"{capital_efficiency:.1%}",
            'Risk Score': f"{(1-survival_rate)*10:.1f}/10"
        })
    
    return pd.DataFrame(results)

# Run analysis
print("Analyzing Collateralization Strategies...")
print("(Monte Carlo simulation with 100 price paths)\n")

strategy_results = analyze_collateralization_strategies()
print(strategy_results.to_string(index=False))

print("\nInterpretation:")
print("- Lower initial ratios = higher capital efficiency but higher liquidation risk")
print("- Consider your risk tolerance when choosing a collateralization strategy")
print("- In volatile markets, conservative ratios (200%+) provide safety margins")

### Challenge 2: Design a More Robust Algorithmic Stablecoin

Based on what you learned about Terra's failure, propose mechanisms that could make algorithmic stablecoins more robust.

In [ ]:
# YOUR TURN: Implement a more robust algorithmic stablecoin simulation

class ImprovedAlgorithmicStablecoin:
    """
    Design an improved algorithmic stablecoin with safeguards.
    
    Consider implementing:
    1. Partial collateralization (like FRAX)
    2. Redemption rate limits (circuit breakers)
    3. Reserve fund for emergencies
    4. Dynamic collateral ratio based on market conditions
    5. Graduated redemption fees during stress
    """
    
    def __init__(
        self,
        collateral_ratio: float = 0.5,  # 50% backed by real collateral
        daily_redemption_cap: float = 0.05,  # Max 5% redeemed per day
        stress_fee: float = 0.05,  # 5% fee during de-peg
        reserve_ratio: float = 0.10  # 10% emergency reserve
    ):
        self.collateral_ratio = collateral_ratio
        self.daily_redemption_cap = daily_redemption_cap
        self.stress_fee = stress_fee
        self.reserve_ratio = reserve_ratio
        
        # TODO: Implement initialization
        pass
    
    def process_redemption(self, amount: float, current_price: float) -> Dict:
        """
        Process a redemption request with safeguards.
        
        TODO: Implement redemption logic with:
        - Rate limiting
        - Stress fees
        - Reserve usage
        """
        pass
    
    def simulate_stress(self, days: int = 14) -> Dict:
        """
        Simulate behavior under stress conditions.
        
        TODO: Run stress test and compare to Terra-style design
        """
        pass

# Design discussion
print("\n" + "=" * 80)
print("CHALLENGE: Design a More Robust Algorithmic Stablecoin")
print("=" * 80)

print("""
Key design questions to consider:

1. COLLATERALIZATION
   - What percentage should be backed by real assets?
   - What types of collateral are acceptable?
   - How to handle collateral volatility?

2. REDEMPTION MECHANISMS
   - Should there be daily redemption limits?
   - Should fees increase during stress?
   - How to prevent bank runs?

3. RESERVES
   - How large should emergency reserves be?
   - What assets should reserves hold?
   - When should reserves be deployed?

4. GOVERNANCE
   - Who can change parameters?
   - How quickly can changes be made?
   - Should there be time locks?

5. TRANSPARENCY
   - How to prove reserves exist?
   - Real-time collateralization dashboards?
   - Audit requirements?

Implement your design above and test it against the Terra collapse scenario!
""")

### Challenge 3: Peg Stability Scoring System

Create a scoring system that rates stablecoin reliability based on historical performance.

In [ ]:
def calculate_stability_score(prices: np.ndarray, name: str) -> Dict:
    """
    Calculate a comprehensive stability score for a stablecoin.
    
    TODO: Implement scoring based on:
    - Average deviation from peg
    - Maximum deviation from peg
    - Time spent outside bands
    - Volatility of deviations
    - Recovery speed after de-peg
    
    Return a score from 0-100 with breakdown.
    """
    deviations = np.abs(prices - 1.0)
    
    # Component scores (each 0-20 points)
    
    # 1. Mean deviation score (lower is better)
    mean_dev = deviations.mean()
    mean_score = max(0, 20 - mean_dev * 1000)
    
    # 2. Max deviation score
    max_dev = deviations.max()
    max_score = max(0, 20 - max_dev * 100)
    
    # 3. Time within 1% band
    within_band = (deviations <= 0.01).sum() / len(prices)
    band_score = within_band * 20
    
    # 4. Volatility score
    volatility = deviations.std()
    vol_score = max(0, 20 - volatility * 500)
    
    # 5. Recovery score (how quickly returns to peg after deviation)
    recovery_score = 15  # Simplified - would need more complex analysis
    
    total_score = mean_score + max_score + band_score + vol_score + recovery_score
    
    # Rating
    if total_score >= 90:
        rating = 'AAA'
    elif total_score >= 80:
        rating = 'AA'
    elif total_score >= 70:
        rating = 'A'
    elif total_score >= 60:
        rating = 'BBB'
    elif total_score >= 50:
        rating = 'BB'
    else:
        rating = 'B'
    
    return {
        'name': name,
        'total_score': total_score,
        'rating': rating,
        'components': {
            'Mean Deviation': mean_score,
            'Max Deviation': max_score,
            'Band Adherence': band_score,
            'Volatility': vol_score,
            'Recovery': recovery_score
        }
    }

# Score our simulated stablecoins
usdc_score = calculate_stability_score(usdc_prices, 'USDC (Fiat-Backed)')
dai_score = calculate_stability_score(dai_prices, 'DAI (Crypto-Backed)')
algo_score = calculate_stability_score(algo_prices, 'Algorithmic')

print("\n" + "=" * 80)
print("STABLECOIN STABILITY SCORES")
print("=" * 80)

for score_data in [usdc_score, dai_score, algo_score]:
    print(f"\n{score_data['name']}")
    print(f"  Overall Score: {score_data['total_score']:.1f}/100")
    print(f"  Rating: {score_data['rating']}")
    print(f"  Component Breakdown:")
    for component, value in score_data['components'].items():
        print(f"    {component}: {value:.1f}/20")

print("\n" + "=" * 80)

## Summary

In this notebook, you learned:

### Key Concepts

1. **Stablecoin Types**: 
   - Fiat-backed (USDT, USDC): Centralized but stable
   - Crypto-backed (DAI): Decentralized but capital inefficient
   - Algorithmic: Capital efficient but high risk

2. **Peg Mechanisms**:
   - Fiat reserves provide direct backing
   - Over-collateralization protects against volatility
   - Algorithmic mechanisms rely on market dynamics

3. **Risk Factors**:
   - Liquidation cascades in crypto-backed systems
   - Death spirals in algorithmic systems
   - Counterparty risk in centralized systems

4. **Lessons from Terra/UST**:
   - Reflexivity can be fatal
   - Market confidence is fragile
   - Unsustainable yields are warning signs

### Practical Takeaways

- **For Users**: Choose stablecoins based on your risk tolerance
  - High safety: USDC (fiat-backed, audited)
  - Decentralization: DAI (crypto-backed)
  - Higher yield but higher risk: Algorithmic options

- **For Developers**: When building with stablecoins:
  - Consider which type fits your use case
  - Implement safeguards for de-peg scenarios
  - Monitor collateralization ratios in real-time

- **For Researchers**: Key open questions:
  - Can algorithmic stablecoins ever be truly stable?
  - What's the optimal collateralization ratio?
  - How to prevent liquidation cascades?

### Further Reading

- [MakerDAO Documentation](https://docs.makerdao.com/) - Understanding DAI
- [Circle's USDC Reserves](https://www.circle.com/en/usdc) - Fiat-backed transparency
- [Terra Post-Mortem](https://www.coindesk.com/learn/the-fall-of-terra-a-timeline-of-the-meteoric-rise-and-crash-of-ust-and-luna/) - Detailed collapse analysis
- [Liquity Protocol](https://www.liquity.org/) - Alternative crypto-backed design
- [DeFi Risk Assessment](https://defillama.com/) - Track stablecoin metrics